In [12]:
import string
import unicodedata
import glob
import os

all_letters = string.ascii_letters
n_letters = len(all_letters)


def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
print(unicode_to_ascii('Ślusàrski'))

Slusarski


In [21]:
def read_file(filename):
    lines = open(filename, encoding='utf-8').read().strip('\n').split('\n')
    return [unicode_to_ascii(line) for line in lines]

all_names_by_category = dict()

for filename in glob.glob('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    lines = read_file(filename)
    all_names_by_category[category] = lines

categories = list(all_names_by_category.keys())
print(categories)

['Arabic', 'Chinese', 'Czech', 'Dutch', 'English', 'French', 'German', 'Greek', 'Irish', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Scottish', 'Spanish', 'Vietnamese']


In [24]:
import torch

def line_to_tensor(s):
    tensor = torch.zeros(len(s), 1, n_letters)
    for i, l in enumerate(s):
        index = all_letters.find(l)
        tensor[i][0][index] = 1
    return tensor

print(line_to_tensor('yi'))

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.]]])


In [30]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(categories)
    line = randomChoice(all_names_by_category[category])
    category_tensor = torch.tensor([categories.index(category)], dtype=torch.long)
    line_tensor = line_to_tensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, category_tensor, '/ line =', line)

category = Greek tensor([7]) / line = Milionis
category = Vietnamese tensor([17]) / line = Chau
category = Czech tensor([2]) / line = Spoerl
category = Irish tensor([8]) / line = Naoimhin
category = Greek tensor([7]) / line = Frangopoulos
category = Greek tensor([7]) / line = Close
category = German tensor([6]) / line = Bosch
category = Scottish tensor([15]) / line = Craig
category = Vietnamese tensor([17]) / line = Do
category = German tensor([6]) / line = Bergfalk


In [47]:
import torch.nn as nn


class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    
    def forward(self, input, hidden):
        combined = torch.cat([input, hidden], 1)
        hidden = self.i2h(combined)
        output = self.softmax(self.i2o(combined))
        return output, hidden
    
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
        
    
n_hidden = 128

rnn = RNN(n_letters, n_hidden, len(categories))

In [48]:
def category_from_output(output):
    _, top_i = output.topk(1)
    category_i = top_i[0].item()
    return categories[category_i], category_i


criterion = nn.NLLLoss()

learning_rate = 0.005

def train(category_tensor, line_tensor):
    hidden = rnn.init_hidden()
    
    rnn.zero_grad()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    loss = criterion(output, category_tensor)
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)
    
    return output, loss.item()

In [50]:
n_iters = 10000

for i in range(n_iters):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    print('%d iter, loss: %.4f'%(i, loss))

0 iter, loss: 2.9450
1 iter, loss: 2.8237
2 iter, loss: 2.7890
3 iter, loss: 2.9668
4 iter, loss: 2.7806
5 iter, loss: 2.8125
6 iter, loss: 2.8195
7 iter, loss: 2.7318
8 iter, loss: 2.9876
9 iter, loss: 3.0210
10 iter, loss: 2.9178
11 iter, loss: 2.8876
12 iter, loss: 2.8714
13 iter, loss: 2.7773
14 iter, loss: 2.8660
15 iter, loss: 2.8765
16 iter, loss: 2.7263
17 iter, loss: 2.9376
18 iter, loss: 2.9079
19 iter, loss: 2.8840
20 iter, loss: 2.6945
21 iter, loss: 2.8488
22 iter, loss: 2.7229
23 iter, loss: 2.8029
24 iter, loss: 2.9055
25 iter, loss: 2.6770
26 iter, loss: 2.6101
27 iter, loss: 2.6811
28 iter, loss: 2.8014
29 iter, loss: 2.7634
30 iter, loss: 2.7175
31 iter, loss: 3.0154
32 iter, loss: 2.7267
33 iter, loss: 2.9607
34 iter, loss: 2.8197
35 iter, loss: 2.9204
36 iter, loss: 2.8517
37 iter, loss: 2.6495
38 iter, loss: 2.8855
39 iter, loss: 2.7943
40 iter, loss: 2.7718
41 iter, loss: 2.8202
42 iter, loss: 2.7454
43 iter, loss: 2.8531
44 iter, loss: 2.7904
45 iter, loss: 2.904

389 iter, loss: 2.7101
390 iter, loss: 2.6097
391 iter, loss: 2.9482
392 iter, loss: 2.8252
393 iter, loss: 2.8723
394 iter, loss: 2.7364
395 iter, loss: 2.8880
396 iter, loss: 2.8788
397 iter, loss: 2.7502
398 iter, loss: 2.6087
399 iter, loss: 2.5980
400 iter, loss: 2.8832
401 iter, loss: 2.7789
402 iter, loss: 2.6685
403 iter, loss: 2.5569
404 iter, loss: 2.8592
405 iter, loss: 2.7707
406 iter, loss: 2.6897
407 iter, loss: 2.6190
408 iter, loss: 2.8344
409 iter, loss: 2.8227
410 iter, loss: 2.8722
411 iter, loss: 2.8077
412 iter, loss: 2.7941
413 iter, loss: 2.9050
414 iter, loss: 2.7809
415 iter, loss: 2.7564
416 iter, loss: 2.9292
417 iter, loss: 2.8884
418 iter, loss: 2.8502
419 iter, loss: 2.9897
420 iter, loss: 2.9140
421 iter, loss: 2.9058
422 iter, loss: 2.9558
423 iter, loss: 2.5962
424 iter, loss: 2.6712
425 iter, loss: 2.7179
426 iter, loss: 2.8458
427 iter, loss: 2.7364
428 iter, loss: 2.8134
429 iter, loss: 2.7597
430 iter, loss: 2.8433
431 iter, loss: 2.8677
432 iter, l

762 iter, loss: 2.7510
763 iter, loss: 2.7954
764 iter, loss: 2.8981
765 iter, loss: 2.9316
766 iter, loss: 2.9348
767 iter, loss: 3.0005
768 iter, loss: 2.8279
769 iter, loss: 2.8984
770 iter, loss: 2.9161
771 iter, loss: 2.7417
772 iter, loss: 2.7070
773 iter, loss: 2.6972
774 iter, loss: 2.8679
775 iter, loss: 2.8770
776 iter, loss: 2.4672
777 iter, loss: 2.7715
778 iter, loss: 2.5939
779 iter, loss: 2.8213
780 iter, loss: 2.5860
781 iter, loss: 2.9344
782 iter, loss: 2.7424
783 iter, loss: 2.7469
784 iter, loss: 2.8586
785 iter, loss: 2.8952
786 iter, loss: 2.7948
787 iter, loss: 2.9965
788 iter, loss: 2.7484
789 iter, loss: 2.7964
790 iter, loss: 2.8850
791 iter, loss: 2.8446
792 iter, loss: 2.8816
793 iter, loss: 2.7181
794 iter, loss: 2.8027
795 iter, loss: 2.9796
796 iter, loss: 2.9722
797 iter, loss: 2.7965
798 iter, loss: 2.9214
799 iter, loss: 2.9049
800 iter, loss: 2.4263
801 iter, loss: 2.5507
802 iter, loss: 2.7513
803 iter, loss: 2.8431
804 iter, loss: 2.7313
805 iter, l

1128 iter, loss: 2.8016
1129 iter, loss: 2.7141
1130 iter, loss: 2.7377
1131 iter, loss: 2.9817
1132 iter, loss: 3.0456
1133 iter, loss: 2.8860
1134 iter, loss: 2.8626
1135 iter, loss: 2.2483
1136 iter, loss: 2.8581
1137 iter, loss: 2.7259
1138 iter, loss: 3.0089
1139 iter, loss: 2.8578
1140 iter, loss: 2.5032
1141 iter, loss: 3.0431
1142 iter, loss: 2.8480
1143 iter, loss: 2.6145
1144 iter, loss: 2.8213
1145 iter, loss: 3.0737
1146 iter, loss: 2.8110
1147 iter, loss: 3.0338
1148 iter, loss: 2.7108
1149 iter, loss: 2.7715
1150 iter, loss: 2.8500
1151 iter, loss: 2.9577
1152 iter, loss: 2.7661
1153 iter, loss: 2.9199
1154 iter, loss: 2.7527
1155 iter, loss: 2.8044
1156 iter, loss: 2.7096
1157 iter, loss: 2.8358
1158 iter, loss: 2.6447
1159 iter, loss: 2.7603
1160 iter, loss: 2.7782
1161 iter, loss: 2.9896
1162 iter, loss: 2.6231
1163 iter, loss: 2.8067
1164 iter, loss: 2.6362
1165 iter, loss: 2.9107
1166 iter, loss: 2.6996
1167 iter, loss: 2.7429
1168 iter, loss: 2.9620
1169 iter, loss:

1527 iter, loss: 2.9439
1528 iter, loss: 2.8570
1529 iter, loss: 2.9505
1530 iter, loss: 2.8398
1531 iter, loss: 2.8113
1532 iter, loss: 2.7680
1533 iter, loss: 2.8754
1534 iter, loss: 2.5677
1535 iter, loss: 2.8536
1536 iter, loss: 2.7210
1537 iter, loss: 2.7404
1538 iter, loss: 2.9824
1539 iter, loss: 2.8297
1540 iter, loss: 2.7778
1541 iter, loss: 2.6310
1542 iter, loss: 2.8371
1543 iter, loss: 2.8074
1544 iter, loss: 2.6272
1545 iter, loss: 3.0149
1546 iter, loss: 2.9975
1547 iter, loss: 2.7987
1548 iter, loss: 2.8577
1549 iter, loss: 2.8096
1550 iter, loss: 2.7034
1551 iter, loss: 2.3468
1552 iter, loss: 2.4243
1553 iter, loss: 2.6458
1554 iter, loss: 2.9390
1555 iter, loss: 2.7178
1556 iter, loss: 2.5679
1557 iter, loss: 2.7210
1558 iter, loss: 2.9600
1559 iter, loss: 2.3700
1560 iter, loss: 2.6733
1561 iter, loss: 2.5612
1562 iter, loss: 2.5845
1563 iter, loss: 2.8407
1564 iter, loss: 2.9946
1565 iter, loss: 2.8607
1566 iter, loss: 2.8555
1567 iter, loss: 2.7599
1568 iter, loss:

...

8163 iter, loss: 2.1794
8164 iter, loss: 1.6995
8165 iter, loss: 2.9808
8166 iter, loss: 1.5359
8167 iter, loss: 3.7036
8168 iter, loss: 2.0241
8169 iter, loss: 2.9385
8170 iter, loss: 0.3741
8171 iter, loss: 2.8020
8172 iter, loss: 2.5846
8173 iter, loss: 1.5149
8174 iter, loss: 4.4298
8175 iter, loss: 2.1793
8176 iter, loss: 2.0172
8177 iter, loss: 2.1329
8178 iter, loss: 2.7339
8179 iter, loss: 2.0314
8180 iter, loss: 1.5920
8181 iter, loss: 1.9372
8182 iter, loss: 2.7735
8183 iter, loss: 2.3975
8184 iter, loss: 0.4786
8185 iter, loss: 1.9173
8186 iter, loss: 1.7286
8187 iter, loss: 2.7144
8188 iter, loss: 1.8261
8189 iter, loss: 2.3102
8190 iter, loss: 2.7792
8191 iter, loss: 2.2428
8192 iter, loss: 1.0839
8193 iter, loss: 1.5897
8194 iter, loss: 2.0392
8195 iter, loss: 2.4229
8196 iter, loss: 1.6834
8197 iter, loss: 1.1201
8198 iter, loss: 2.7601
8199 iter, loss: 2.5036
8200 iter, loss: 3.3274
8201 iter, loss: 2.3295
8202 iter, loss: 1.7842
8203 iter, loss: 1.5500
8204 iter, loss:

8573 iter, loss: 2.3423
8574 iter, loss: 1.7731
8575 iter, loss: 1.1143
8576 iter, loss: 2.2555
8577 iter, loss: 1.4036
8578 iter, loss: 2.2682
8579 iter, loss: 2.9427
8580 iter, loss: 2.3025
8581 iter, loss: 2.7488
8582 iter, loss: 1.4831
8583 iter, loss: 1.8926
8584 iter, loss: 1.9539
8585 iter, loss: 1.5814
8586 iter, loss: 2.1572
8587 iter, loss: 2.3259
8588 iter, loss: 2.2722
8589 iter, loss: 1.5609
8590 iter, loss: 1.5438
8591 iter, loss: 2.3741
8592 iter, loss: 1.0249
8593 iter, loss: 1.5140
8594 iter, loss: 3.4738
8595 iter, loss: 2.3199
8596 iter, loss: 2.1106
8597 iter, loss: 2.4274
8598 iter, loss: 1.9303
8599 iter, loss: 2.0792
8600 iter, loss: 3.8722
8601 iter, loss: 2.3484
8602 iter, loss: 2.2010
8603 iter, loss: 0.7183
8604 iter, loss: 2.9987
8605 iter, loss: 3.7054
8606 iter, loss: 2.2869
8607 iter, loss: 1.3329
8608 iter, loss: 2.3472
8609 iter, loss: 2.5083
8610 iter, loss: 0.9969
8611 iter, loss: 2.2654
8612 iter, loss: 1.8783
8613 iter, loss: 1.9382
8614 iter, loss:

8931 iter, loss: 1.7407
8932 iter, loss: 1.0803
8933 iter, loss: 2.9542
8934 iter, loss: 2.5425
8935 iter, loss: 1.6886
8936 iter, loss: 1.9413
8937 iter, loss: 2.0008
8938 iter, loss: 2.1398
8939 iter, loss: 2.2773
8940 iter, loss: 2.2102
8941 iter, loss: 3.4534
8942 iter, loss: 3.2189
8943 iter, loss: 0.9039
8944 iter, loss: 0.2848
8945 iter, loss: 2.7925
8946 iter, loss: 1.4342
8947 iter, loss: 1.6613
8948 iter, loss: 2.2307
8949 iter, loss: 2.3854
8950 iter, loss: 1.9279
8951 iter, loss: 1.3749
8952 iter, loss: 1.8745
8953 iter, loss: 1.6843
8954 iter, loss: 2.2491
8955 iter, loss: 1.6009
8956 iter, loss: 2.4173
8957 iter, loss: 2.5197
8958 iter, loss: 2.9257
8959 iter, loss: 2.7544
8960 iter, loss: 1.9598
8961 iter, loss: 0.9376
8962 iter, loss: 2.8813
8963 iter, loss: 3.0410
8964 iter, loss: 0.9340
8965 iter, loss: 2.6463
8966 iter, loss: 2.4084
8967 iter, loss: 2.1241
8968 iter, loss: 1.6768
8969 iter, loss: 1.3941
8970 iter, loss: 1.0888
8971 iter, loss: 4.4174
8972 iter, loss:

9307 iter, loss: 1.6636
9308 iter, loss: 3.2410
9309 iter, loss: 1.1069
9310 iter, loss: 2.3306
9311 iter, loss: 2.8544
9312 iter, loss: 2.1263
9313 iter, loss: 1.1138
9314 iter, loss: 2.2429
9315 iter, loss: 1.4517
9316 iter, loss: 3.2473
9317 iter, loss: 2.2065
9318 iter, loss: 1.5461
9319 iter, loss: 2.1540
9320 iter, loss: 3.3977
9321 iter, loss: 1.4217
9322 iter, loss: 2.3962
9323 iter, loss: 2.9376
9324 iter, loss: 1.5554
9325 iter, loss: 1.3401
9326 iter, loss: 1.8112
9327 iter, loss: 3.0626
9328 iter, loss: 1.4070
9329 iter, loss: 2.1487
9330 iter, loss: 1.2709
9331 iter, loss: 3.4694
9332 iter, loss: 2.2824
9333 iter, loss: 2.7013
9334 iter, loss: 1.8184
9335 iter, loss: 2.4280
9336 iter, loss: 2.1178
9337 iter, loss: 1.3881
9338 iter, loss: 3.1262
9339 iter, loss: 1.9713
9340 iter, loss: 1.0813
9341 iter, loss: 1.9869
9342 iter, loss: 1.6960
9343 iter, loss: 1.0297
9344 iter, loss: 1.7566
9345 iter, loss: 1.9191
9346 iter, loss: 2.1427
9347 iter, loss: 1.0347
9348 iter, loss:

9653 iter, loss: 2.8868
9654 iter, loss: 2.2830
9655 iter, loss: 2.9852
9656 iter, loss: 0.8757
9657 iter, loss: 2.1906
9658 iter, loss: 1.1381
9659 iter, loss: 2.5028
9660 iter, loss: 1.4725
9661 iter, loss: 3.1546
9662 iter, loss: 2.0641
9663 iter, loss: 3.4042
9664 iter, loss: 1.5155
9665 iter, loss: 2.2591
9666 iter, loss: 2.9412
9667 iter, loss: 2.0185
9668 iter, loss: 1.5570
9669 iter, loss: 3.2539
9670 iter, loss: 2.7403
9671 iter, loss: 1.6353
9672 iter, loss: 1.0747
9673 iter, loss: 3.1223
9674 iter, loss: 2.7958
9675 iter, loss: 0.3116
9676 iter, loss: 1.7341
9677 iter, loss: 1.5042
9678 iter, loss: 1.2080
9679 iter, loss: 1.8040
9680 iter, loss: 1.7747
9681 iter, loss: 4.2309
9682 iter, loss: 1.9480
9683 iter, loss: 2.7484
9684 iter, loss: 2.3456
9685 iter, loss: 2.1570
9686 iter, loss: 2.7547
9687 iter, loss: 2.2861
9688 iter, loss: 2.8269
9689 iter, loss: 1.1110
9690 iter, loss: 3.0067
9691 iter, loss: 2.9775
9692 iter, loss: 2.1972
9693 iter, loss: 1.8890
9694 iter, loss:

In [51]:
def predict(input_name, n_prediction=3):
    print('\n> %s'%(input_name))
    with torch.no_grad():
        hidden = rnn.init_hidden()
        line_tensor = line_to_tensor(input_name)
        for i in range(len(input_name)):
            output, hidden = rnn(line_tensor[i], hidden)
        topv, topi = output.topk(n_prediction, 1, True)
        
        for i in range(n_prediction):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('%.2f %s'%(value, categories[category_index]))

In [66]:
predict('jane')


> jane
-1.69 Chinese
-1.74 Vietnamese
-1.77 Korean
